In [1]:
# Goal of this notebook is to create a set of tcl scripts that can be used to generate vivado_hls projects, and then run synthesis and evntualy export the IP to vivado and run the implementation and bitstream generation.


def create_project(
    project_name,
    project_dir,
    top_function,
    src_files,
    inc_dirs,
    part,
    clock_period,
    language="vhdl",
    board="zedboard",
):
    """
    Create a vivado_hls project with the given parameters.
    """
    # Create the project directory
    os.system("mkdir -p {}".format(project_dir))
    # Create the project script
    project_script = """\
open_project -reset {}
set_top {}
add_files {}
set_part {}
create_clock -period {} -name default
""".format(
        project_name, top_function, " ".join(src_files), part, clock_period
    )
    # Add the include directories
    for inc_dir in inc_dirs:
        project_script += "add_include_dir {}\n".format(inc_dir)
    # Add the language
    project_script += "set_top_lang {}\n".format(language)
    # Add the board
    project_script += "set_board {}\n".format(board)
    # Add the synthesis script
    project_script += """\
csynth_design
exit
"""
    # Write the project script
    with open(os.path.join(project_dir, "project.tcl"), "w") as f:
        f.write(project_script)
    # Create the run script
    run_script = """\
vivado_hls -f project.tcl
"""
    # Write the run script
    with open(os.path.join(project_dir, "run.sh"), "w") as f:
        f.write(run_script)
    # Make the run script executable
    os.system("chmod +x {}".format(os.path.join(project_dir, "run.sh")))
    # Create the vivado_hls project
    os.system("cd {} && ./run.sh".format(project_dir))

In [2]:
create_project("test", "test", "test", ["test.vhd"], ["test"], "xc7z020clg484-1", 10.0)

./run.sh: 1: vivado_hls: not found


In [5]:
import string
import os
import sys

template_tcl_script = "template_script.tcl"
tcl_dict = {
    "project_name": "vm",
    "top": "VM_INT8_V2_0",
    "part": "xc7z020clg400-1",
    "solution": "current_solution",
    "period": "5",
    "vendor": "xilinx",
}
with open(template_tcl_script) as f:
    tcl_script = str(string.Template(f.read()).substitute(tcl_dict))

with open("script.tcl", "w") as f:
    f.write(tcl_script)

In [45]:
import os
import string
from string import Template

class mt(Template):
    delimiter = '&'
    idpattern = r'[a-z][_a-z0-9]*'

class hardware_exp:
    def __init__(self, acc_name, acc_version, acc_sub_version, acc_test, acc_test_id,acc_test_desc,acc_link_folder, vivado_path):
        self.acc_name = acc_name
        self.acc_version = acc_version
        self.acc_sub_version = acc_sub_version
        self.acc_test = acc_test
        self.acc_test_id = ("_"+ self.acc_test_id) if self.acc_test else ""
        self.acc_test_desc = acc_test_desc
        self.acc_link_folder = acc_link_folder + acc_name +"_src"
        self.acc_tag = str(self.acc_name) + "_" + str(self.acc_version) + "_" + str(self.acc_sub_version) + str(self.acc_test_id)
        self.acc_part = "xc7z020clg400-1"
        self.vp = vivado_path

    def generate_hls_tcl(self, top,output_dir):
        s = ""
        s += "open_project -reset " + self.acc_tag + "\n"
        s += "set_top " + top + "\n"
        for file in os.listdir(self.acc_link_folder):
            if file.endswith(".cc"):
                s += "add_files " + "src/" + file + ' -cflags "-D__SYNTHESIS__"\n'
        
        for file in os.listdir(self.acc_link_folder):
            if file.endswith(".h"):
                s += "add_files " + "src/" + file + '\n'

        s += 'open_solution "' + self.acc_tag + '"\n'
        s += "set_part " + self.acc_part + "\n"
        s += "create_clock -period " + "5" + " -name default\n"
        s += "config_export -format ip_catalog -rtl verilog -taxonomy /s -vendor xilinx\n"
        s += "csynth_design\n"
        s += "export_design -format ip_catalog\n"
        s += "exit\n"

        with open(f"{output_dir}/hls_script.tcl", "w") as f:
            f.write(s)

    def generate_hlx_tcl(self, output_dir, hlx_tcl_script,top):
        with open((hlx_tcl_script)) as f:
            tcl_script = str(mt(f.read()).substitute({"top":top}))
        with open(f"{output_dir}/hlx_script.tcl", "w") as f:
            f.write(tcl_script)

    def create_run_script(self, output_path, hlx_tcl_script):
        output_dir = output_path + "/" + self.acc_tag
        run_script = "#!/bin/bash\n"
        run_script += "start_hls=`date +%s.%N`\n"
        run_script += f"{self.vp}/vivado_hls -f hls_script.tcl\n"
        run_script += "end_hls=`date +%s.%N`\n"
        run_script += f"{self.vp}/vivado -mode batch -source hlx_script.tcl -tclargs --origin_dir . --project_name {self.acc_tag}_hlx --ip_repo ./{self.acc_tag}/{self.acc_tag}\n"
        run_script += "end_hlx=`date +%s.%N`\n"
        run_script += "mkdir -p ./generated_files\n"
        run_script += f"cp ./{self.acc_tag}_hlx/{self.acc_tag}_hlx.srcs/sources_1/bd/design_1/hw_handoff/design_1.hwh ./generated_files/{self.acc_tag}.hwh\n"
        run_script += f"cp ./{self.acc_tag}_hlx/{self.acc_tag}_hlx.runs/impl_1/design_1_wrapper.bit ./generated_files/{self.acc_tag}.bit\n"
        run_script += f"cp ./{self.acc_tag}_hlx/utilization_report_impl_full.txt ./generated_files/\n"
        run_script += f"cp ./{self.acc_tag}_hlx/utilization_report_impl_ip.txt ./generated_files/\n"
        run_script += f"cp ./{self.acc_tag}_hlx/timing_report_impl_full.txt ./generated_files/\n"
        run_script += f"cp ./{self.acc_tag}_hlx/timing_report_impl_ip.txt ./generated_files/\n"
        run_script += f"hls_runtime=$((start_hls-end_hls))\n"
        run_script += f"hlx_runtime=$((end_hlx-end_hls))\n"
        run_script += f"echo HLS runtime: $hls_runtime\n"
        run_script += f"echo HLX runtime: $hlx_runtime\n"

        with open(f"{output_dir}/run.sh", "w") as f:
            f.write(run_script)
        os.system(f"chmod +x {output_dir}/run.sh")
        
    def create_project(self, top, output_path, hlx_tcl_script):
        output_dir = output_path + "/" + self.acc_tag
        output_acc_src_dir = output_dir + "/src"
        os.system(f"mkdir -p {output_dir}")

        # Copy .h and .cc files from the link folder to the output folder
        os.system(f"mkdir -p {output_acc_src_dir}")
        os.system(f"cp {self.acc_link_folder}/*.h {output_acc_src_dir}")
        os.system(f"cp {self.acc_link_folder}/*.cc {output_acc_src_dir}")
        self.generate_hls_tcl(top,output_dir)
        self.generate_hlx_tcl(output_dir, hlx_tcl_script,top)
        self.create_run_script(output_path, hlx_tcl_script)

  

In [46]:
# Hardware variables

acc_name = "vm"
acc_version = 2
acc_sub_version = 0
acc_test = False
acc_test_id = 0
acc_test_desc = "test_0"
acc_link_folder = "./secda_tflite_acc_srcs/"
vivado_path = "/home/jude/Xilinx_2019.2/Vivado/2019.2/bin/"

h1 = hardware_exp(acc_name, acc_version, acc_sub_version, acc_test, acc_test_id,acc_test_desc,acc_link_folder, vivado_path)
h1.create_project("VM_INT8_V2_0", "generated", "vm_hlx.tcl")

